# 定義

In [ ]:
$vmName = 'Test-JupyterPowerShell'
$vmSwitchName = 'Default Switch'
$cred = New-Object System.Management.Automation.PSCredential (
    'sakaztk',
    (ConvertTo-SecureString -AsPlainText -Force -String 'Passw0rd!')
)
$download = @{
    #Uri  = 'https://github.com/sakaztk/pwsh-notebook/raw/master/Anaconda/Anaconda.ps1'
    #Uri  = 'https://github.com/sakaztk/pwsh-notebook/raw/master/Miniconda/Miniconda.ps1'
    #Uri  = 'https://github.com/sakaztk/pwsh-notebook/raw/master/Miniforge/Miniforge.ps1'
    Uri  = 'https://github.com/sakaztk/pwsh-notebook/raw/master/PythonForWindows/PythonForWindows.ps1'
    OutFile = 'install.ps1'
}
$scriptOpt = @{
    Verbose = $true
    CleanupDownloadFiles = $true
    InstallPwsh7SDK = $true
    InstallDotnetInteractive = $true
}

# 前処理

## WinRM設定

In [ ]:
$winrm = Get-Service WinRM
if ($winrm.Status -eq 'Stopped' -and $winrm.StartType -eq 'Manual') {
    winrm quickconfig -force
} 

## チェックポイント作成

In [ ]:
Checkpoint-VM -Name $vmName -SnapshotName 'BeforeTests' -Verbose -Passthru

## VM起動

In [ ]:
Start-VM -Name $vmName

In [ ]:
$retry = 0
while ($retry -lt 300) {
    try {
        $dsIpAddress = (Get-VMNetworkAdapter -VMName $vmName | Where-Object {$_.SwitchName -eq 'Default Switch'}).IPAddresses |
        Where-Object {($_ -match '^\d{1,3}(\.\d{1,3}){3}$') -and ($_ -notlike '169.254*') -and ($_ -ne '0.0.0.0')}
        Test-WSMan $dsIpAddress -ErrorAction Stop
        break
    } catch {
        Start-Sleep -Seconds 10
        $retry++
    }
}

## IPアドレス取得

In [ ]:
$dsIpAddress = (Get-VMNetworkAdapter -VMName $vmName | Where-Object {$_.SwitchName -eq 'Default Switch'}).IPAddresses |
    Where-Object {($_ -match '^\d{1,3}(\.\d{1,3}){3}$') -and ($_ -notlike '169.254*') -and ($_ -ne '0.0.0.0')}
$dsIpAddress

## TrustedHosts設定

In [ ]:
$current = (Get-Item WSMan:\localhost\Client\TrustedHosts).Value
$currentList = if ($current) { $current -split ',' | ForEach-Object { $_.Trim() } } else { @() }
if ($currentList -notcontains $dsIpAddress -and $current -ne '*') {
    $newValue = @($currentList) + @($dsIpAddress) -join ','
    Set-Item WSMan:\localhost\Client\TrustedHosts -Value $newValue -Force -Verbose
}
Get-Item WSMan:\localhost\Client\TrustedHosts

## PSSession作成

In [ ]:
$psSession = New-PSSession -ComputerName $dsIpAddress -Credential $cred -Verbose
$psSession

# インストール

## HOMEPATHに移動

In [ ]:
Invoke-Command -Session $psSession -ScriptBlock {
    Set-Location $env:HOMEPATH -Verbose -PassThru
}

## ダウンロード

In [ ]:
Invoke-Command -Session $psSession -ScriptBlock {
    Invoke-WebRequest -UseBasicParsing -Verbose @Using:download
    Get-Item *.ps1
}

## ExecutionPolicy

In [ ]:
Invoke-Command -Session $psSession -ScriptBlock {
    Set-ExecutionPolicy Bypass -Scope Process -Force -Verbose
    Get-ExecutionPolicy
}

## スクリプト実行

In [ ]:
Invoke-Command -Session $psSession -ScriptBlock {
    & .\$($Using:download.OutFile) @Using:scriptOpt
} -Verbose | ForEach-Object { Write-Host $_ }

# 後処理

## 切断

In [ ]:
$psSession | Remove-PSSession

## VM停止

In [ ]:
Stop-VM -Name $vmName -TurnOff

## チェックポイントリストア

In [ ]:
Restore-VMSnapshot -VMName $vmName -Name 'BeforeTests' -Confirm:$false -Verbose

## チェックポイント削除

In [ ]:
Remove-VMSnapshot -VMName $vmName -Name 'BeforeTests'

## TrustedHostsから削除

In [ ]:
$current = (Get-Item WSMan:\localhost\Client\TrustedHosts).Value
if ($current -and $current -ne '*' -and $current -ne '') {
    $hostList = $current -split "," | ForEach-Object { $_.Trim() }
    if ($hostList -contains $dsIpAddress) {
        $updatedList = ($hostList | Where-Object { $_ -ne $dsIpAddress }) -join ","
        Set-Item WSMan:\localhost\Client\TrustedHosts -Value $updatedList -Force
    }
}
Get-Item WSMan:\localhost\Client\TrustedHosts